In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import nltk
from nltk.corpus import stopwords
import string
from sklearn.metrics.pairwise import cosine_similarity
import os
from tqdm import tqdm

from sentence_transformers import SentenceTransformer, util


C:\Users\q\anaconda3\envs\pytorch_env\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
df_train = pd.read_csv('../data/df_train_10percent.csv')
df_test = pd.read_csv('../data/df_test.csv')

In [5]:
df_train

,query,pos_0,pos_1,pos_2,pos_3,pos_4,neg_0,neg_1,neg_2,neg_3,neg_4
0,клинический переулок название улица различный ...,переулок название клинический расположить разн...,различный город бывший ссср улица название кли...,улица клинический переулок встретить разный на...,улица название клинический переулок пролегать ...,название клинический переулок использоваться у...,бывший ссср иметь улица название клинический п...,клинический переулок улица который различный н...,город бывший ссср существовать улица название ...,название клинический переулок использоваться у...,улица различный населить пункт бывший ссср наз...
1,также смотреть схема развитие сеть городской т...,изучить также план развитие городской транспор...,посмотреть схема развитие городской транспорт ...,приглядеться план развитие городской транспорт...,рассмотреть также план развитие городской тран...,взглянуть схема развитие городской транспорт н...,обращать внимание схема развитие сеть городско...,игнорировать план развитие городской транспорт...,следовать схема развитие городской транспорт н...,учитывать план развитие городской транспорт но...,принимать внимание схема развитие городской тр...
2,окончание московский среднее технический учили...,работать электромеханик московский практически...,окончание московский среднее технический учили...,электромеханик московский практический электро...,окончание московский среднее технический учили...,электромеханик московский практический электро...,окончить московский средний технический училищ...,окончание московский среднее технический учили...,окончание московский среднее технический учили...,работать электромеханик московский практически...,окончание московский среднее технический учили...
3,благодаря промежуточный клетка каждый часть те...,каждый часть тело хотя один промежуточный клет...,благодаря промежуточный клетка каждый часть те...,каждый часть тело хотя один промежуточный клет...,образование новый гидра возможно каждый часть ...,наличие хотя один промежуточный клетка каждый ...,один часть тело способный образовать новый гид...,отсутствие промежуточный клетка каждый часть т...,отсутствие хотя один промежуточный клетка част...,промежуточный клетка каждый часть тело образов...,наличие промежуточный клетка часть тело исключ...
4,объект входить число перечисленный оригинальны...,оригинальный редакция новый общий каталог объе...,объект включить список оригинальный версия нов...,являться часть перечень указанный первоначальн...,объект указать оригинальный список новый общий...,входить число объект перечисленный первоначаль...,объект включить список перечисленный оригиналь...,оригинальный версия новый общий каталог объект...,входить число объект перечисленный оригинальны...,объект исключить список оригинальный редакция ...,упомянуть первоначальный версия новый общий ка...
...,...,...,...,...,...,...,...,...,...,...,...
9995,вальобазиликата коммуна италия располагаться р...,коммуна вальобазиликата расположить италия рег...,регион базиликата провинция потенца находиться...,итальянский италия располагаться коммуна вальо...,коммуна вальобазиликата находиться италия пров...,провинция потенца регион базиликата расположит...,вальобазиликата коммуна италия располагаться р...,вальобазиликата коммуна франция располагаться ...,милан коммуна италия располагаться регион ломб...,потенца коммуна италия располагаться регион си...,рим столица италия располагаться регион лацио ...
9996,венок сонет сергей калугинмузыкальный миниатюр...,сергей калугин написать сборник венок сонет,алексей бурков создать музыкальный миниатюра,игорь деревянко эвелина шмелёв придумать конце...,эвелина шмелёв заниматься запись сведение,дмитрий воронцов заниматься художественный офо...,венок сонет иметь отношение сергей калугин,алексей бурков создавать музыкальный миниатюра,игорь деревянко эвелина шмелёв задумывать конц...,запись сведение выполнить эвелина шмелёв,дмитрий воронцов заниматься художественный офо...
9997,расстояние близкий железнодорожный станция сер..

In [8]:
# Инициализируем модель
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

def process_dataframe(df, pos_prefix='pos_', neg_prefix='neg_', n_pos=5, n_neg=5):
    """
    Обрабатывает DataFrame, вычисляет сходство между 'query' и 'pos'/'neg' текстами.
    
    Параметры:
    - df: исходный DataFrame.
    - pos_prefix: префикс для положительных колонок (по умолчанию 'pos_').
    - neg_prefix: префикс для отрицательных колонок (по умолчанию 'neg_').
    - n_pos: количество положительных колонок (по умолчанию 5).
    - n_neg: количество отрицательных колонок (по умолчанию 5).
    
    Возвращает:
    - final_df: преобразованный DataFrame с колонками 'text1', 'text2', 'score'.
    """
    # Определяем названия колонок
    pos_columns = [f'{pos_prefix}{i}' for i in range(n_pos)]
    neg_columns = [f'{neg_prefix}{i}' for i in range(n_neg)]
    
    # Преобразуем положительные примеры
    df_pos = df.melt(
        id_vars=['query'],
        value_vars=pos_columns,
        var_name='type',
        value_name='text'
    )
    df_pos['label'] = 'pos'
    
    # Преобразуем отрицательные примеры
    df_neg = df.melt(
        id_vars=['query'],
        value_vars=neg_columns,
        var_name='type',
        value_name='text'
    )
    df_neg['label'] = 'neg'
    
    # Объединяем положительные и отрицательные примеры
    df_long = pd.concat([df_pos, df_neg], ignore_index=True)
    
    # Удаляем возможные пропуски
    df_long.dropna(subset=['query', 'text'], inplace=True)
    
    # Переименовываем колонки для удобства
    df_long.rename(columns={'query': 'text1', 'text': 'text2'}, inplace=True)
    
    # Удаляем временные колонки
    df_long.drop(columns=['type', 'label'], inplace=True)
    
    # Получаем уникальные запросы и тексты
    unique_queries = df_long['text1'].unique()
    unique_texts = df_long['text2'].unique()
    
    print(f"Уникальных query: {len(unique_queries)}")
    print(f"Уникальных текстов: {len(unique_texts)}")
    
    # Кодируем запросы
    print("Кодирование запросов...")
    query_embeddings = model.encode(unique_queries, convert_to_tensor=True, show_progress_bar=True)
    
    # Кодируем тексты
    print("Кодирование текстов...")
    text_embeddings = model.encode(unique_texts, convert_to_tensor=True, show_progress_bar=True)
    
    # Создаем словари для быстрого доступа
    query_emb_dict = dict(zip(unique_queries, query_embeddings))
    text_emb_dict = dict(zip(unique_texts, text_embeddings))
    
    # Функция для вычисления сходства
    def compute_similarity(row):
        q_emb = query_emb_dict[row['text1']]
        t_emb = text_emb_dict[row['text2']]
        similarity = util.cos_sim(q_emb, t_emb).item()  # Значение от -1 до 1
        scaled_similarity = similarity * 5  # Масштабируем до -5 до 5
        return scaled_similarity
    
    # Применяем функцию ко всем строкам с отображением прогресса
    print("Вычисление сходства...")
    tqdm.pandas()
    df_long['score'] = df_long.progress_apply(compute_similarity, axis=1)
    
    # Создаем итоговый DataFrame
    final_df = df_long[['text1', 'text2', 'score']]
    
    return final_df


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

C:\Users\q\anaconda3\envs\pytorch_env\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\q\.cache\huggingface\hub\models--sentence-transformers--paraphrase-multilingual-MiniLM-L12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
# Обработка df_train
print("Обработка df_train...")
final_train_df = process_dataframe(df_train)

# Обработка df_test
print("Обработка df_test...")
final_test_df = process_dataframe(df_test)


Обработка df_train...
Уникальных query: 9966
Уникальных текстов: 97341
Кодирование запросов...


Batches:   0%|          | 0/312 [00:00<?, ?it/s]

C:\Users\q\anaconda3\envs\pytorch_env\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Кодирование текстов...


Batches:   0%|          | 0/3042 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/63 [00:00<?, ?it/s]

Кодирование текстов...


Batches:   0%|          | 0/610 [00:00<?, ?it/s]

Вычисление сходства...


100%|██████████████████████████████████████████████████████████████████████████| 19999/19999 [00:07<00:00, 2691.18it/s]


In [13]:
# Просмотр первых строк итогового df_train
print("Итоговый DataFrame для df_train:")
final_train_df.head()


Итоговый DataFrame для df_train:


,text1,text2,score
0,клинический переулок название улица различный ...,переулок название клинический расположить разн...,4.217401
1,также смотреть схема развитие сеть городской т...,изучить также план развитие городской транспор...,4.750726
2,окончание московский среднее технический учили...,работать электромеханик московский практически...,4.883469
3,благодаря промежуточный клетка каждый часть те...,каждый часть тело хотя один промежуточный клет...,4.852940
4,объект входить число перечисленный оригинальны...,оригинальный редакция новый общий каталог объе...,4.403906


In [14]:
# Просмотр первых строк итогового df_test
print("Итоговый DataFrame для df_test:")
final_test_df.head()

Итоговый DataFrame для df_test:


,text1,text2,score
0,отвечать работа партия внести свой вклад 1936 ...,внести свой вклад урегулирование инцидент сиан...,4.889081
1,сложный модель поддержка шифрование данные аут...,сложный модель обладать функция шифрование аут...,4.865897
2,это время отплыть два лодка лодка сам генерал ...,два лодка отправиться плавание лодка генерал у...,4.316391
3,первый юридически обязывать добровольный докум...,решение 18551863 добровольный вхождение киргиз...,4.767547
4,длина передний крыло 4552 мм размах крыло 1001...,передний крыло иметь длина 4552 мм размах крыл...,4.839230


In [15]:

# Сохранение итоговых DataFrame
final_train_df.to_csv('../data/final_train_df.csv', index=False)
final_test_df.to_csv('../data/final_test_df.csv', index=False)
